In [10]:
import pickle

with open('细胞系.pkl', 'rb') as file:
    cell_dict_W = pickle.load(file)
    cell_dict_columns = pickle.load(file)
    cell_dict_high_lines = pickle.load(file)

In [11]:
with open('药物对.pkl', 'rb') as file:
    drug_dict_W = pickle.load(file)
    drug_dict_columns = pickle.load(file)
    drug_dict_high_lines = pickle.load(file)

In [12]:
import numpy as np
import pandas as pd
import re,math
from sklearn import preprocessing
from scipy.stats import pearsonr
import matplotlib.pyplot as plt


data1 = pd.read_excel(r'pcbi.1006752.s002.xlsx', sheet_name="Sheet1",index_col='Drug').drop('SMILES',axis=1)
data2 = pd.read_excel(r'Label_synergy4000.xlsx')
data2_columns = data2.columns[2:]

In [13]:
# 结构  {名字： 名字列表， 权重： 权重列表} 顺序对应
dict_molecule_cell = {}
for elem in cell_dict_high_lines:
    dict_molecule_cell[elem] = []
    dict_molecule_cell[elem].append(cell_dict_columns[elem]["max_relevance"][-5:])
    dict_molecule_cell[elem].append(cell_dict_W[elem][-5:])

In [14]:
dict_molecule_drug = {}
for elem in drug_dict_high_lines:
    dict_molecule_drug[elem] = []
    dict_molecule_drug[elem].append(drug_dict_columns[elem]["max_relevance"][-5:])
    dict_molecule_drug[elem].append(drug_dict_W[elem][-5:])

In [15]:
# 拆分索引获取每行对应的drugA和drugB
drugA = []
drugB = []
for i in dict_molecule_drug:
    drugA.append(i.split(", ")[0][1:-1])
    drugB.append(i.split(", ")[1][1:-1])

In [16]:
# 原始数据中删除药物对得分低于0.90的行
temp = []
for index,row in data2.iterrows():
    status = 0
    for i,j in zip(drugA, drugB):
        if i == row[0] and j == row[1]:
            status = 1
    if status == 1:
        temp.append(row)
data22 = pd.DataFrame(temp)

In [17]:
# 获取得分高的细胞系名称组
cells = []
for i in dict_molecule_cell:
    cells.append(i)
cells.extend(data22.columns[0:2])

In [18]:
# 从原始数据中按列取得分高的细胞系
temp = []
for column,line in data22.iteritems():
    if column in cells:
        temp.append(line)
data222 = pd.DataFrame(temp).T

In [19]:
# 筛选结果合并drugA和drugB
tptt = []
for i,j in zip(data222['DrugA'], data222['DrugB']):
    tptt.append(f"'{i}', '{j}'")
data222 = pd.DataFrame(data222.iloc[:,2:].values,index=tptt,columns=data222.columns[2:])

In [20]:
# 原始数据合并drugA和drugB
tptt = []
for i,j in zip(data2['DrugA'], data2['DrugB']):
    tptt.append(f"'{i}', '{j}'")
data3 = pd.DataFrame(data2.iloc[:,2:].values,index=tptt,columns=data2.columns[2:])
data2 = data3.copy()

In [21]:
dict_molecule_cell

{'A2058_SKIN ': [Index(['MDAMB436_BREAST ', 'HT144_SKIN ', 'SKMEL30_SKIN ', 'UACC62_SKIN ',
         'NCIH460_LUNG '],
        dtype='object'),
  array([0.09100395, 0.13110556, 0.14273595, 0.1779505 , 0.22311708])],
 'A2780_OVARY ': [Index(['CAOV3_OVARY ', 'SW620_LARGE_INTESTINE ', 'A2058_SKIN ',
         'VCAP_PROSTATE ', 'COLO320_LARGE_INTESTINE '],
        dtype='object'),
  array([0.07210755, 0.07467228, 0.08339976, 0.22259993, 0.4901396 ])],
 'A375_SKIN ': [Index(['RKO_LARGE_INTESTINE ', 'UACC62_SKIN ', 'MSTO211H_PLEURA ',
         'A2058_SKIN ', 'HT144_SKIN '],
        dtype='object'),
  array([8.68755808e-05, 2.59003372e-04, 1.30224377e-03, 2.43787509e-03,
         6.17920176e-02])],
 'A427_LUNG ': [Index(['COLO320_LARGE_INTESTINE ', 'CAOV3_OVARY ', 'A2780_OVARY ',
         'T47D_BREAST ', 'NCIH23_LUNG '],
        dtype='object'),
  array([0.00152058, 0.00189082, 0.01278501, 0.01666771, 0.12073434])],
 'CAOV3_OVARY ': [Index(['SW837_LARGE_INTESTINE ', 'A2058_SKIN ', 'A2780_OVARY

In [22]:
# 预测筛选后的药物对与细胞系的协同得分
result_df = []
for i in data222.index:
    tp = []
    for k in data222.columns[0:]:
        molecule = 0
        denominator = 0
        a = dict_molecule_cell[k][0]
        b = dict_molecule_drug[i][0]
        cw = dict_molecule_cell[k][1]
        dw = dict_molecule_drug[i][1]
        for m,ccw in zip(a,cw): # 细胞系   名字  权重
            for n,ddw in zip(b,dw): # 药物对   名字   权重
                # 分子子项 = 细胞系权重 × data2对应名字的值 ×药物对权重
                temp_molecule = ccw*data2[data2.index==n][m].values[0]*ddw
                # 分母子项 = 细胞系权重 × 药物对权重
                temp_denominator = ccw*ddw
                molecule += temp_molecule
                denominator += temp_denominator
        result = molecule / denominator
        tp.append(result)
    result_df.append(tp)

In [23]:
sens=pd.DataFrame(np.array(result_df),index = data222.index, columns=data222.columns)

In [24]:
sens

,A2058_SKIN,A2780_OVARY,A375_SKIN,A427_LUNG,CAOV3_OVARY,COLO320_LARGE_INTESTINE,EFM192A_BREAST,ES2_OVARY,HCT116_LARGE_INTESTINE,HT144_SKIN,...,SKMES1_LUNG,SKOV3_OVARY,SW620_LARGE_INTESTINE,SW837_LARGE_INTESTINE,T47D_BREAST,UACC62_SKIN,VCAP_PROSTATE,ZR751_BREAST,DLD1_LARGE_INTESTINE,PA1_OVARY
"'MK-5108', 'SORAFENIB'",2.792456,5.442933,-0.474828,-0.612271,1.454602,5.493418,7.675628,1.114422,-0.577992,6.269505,...,1.810787,-6.595042,4.880433,-0.346719,7.142594,1.608813,7.038133,2.057094,12.781803,1.152360
"'VINORELBINE', 'SUNITINIB'",-1.162660,6.508431,4.314476,-22.333231,-1.124538,-1.300359,-13.058889,1.419484,2.939204,-1.384393,...,-6.732277,-10.609674,-3.143610,6.527121,-11.222961,1.768039,1.379531,1.866333,-10.228760,6.750134
"'SUNITINIB', 'MK-8776'",1.778296,6.485968,-0.313071,-1.066113,-0.804405,2.659350,12.883357,0.044612,1.451485,2.807492,...,6.313088,-4.577809,6.230198,3.068863,11.203059,-1.030967,4.144291,6.038622,3.219628,2.348631
"'5-FU', 'DINACICLIB'",-3.693211,-0.135308,-14.218138,-4.135773,-5.316139,-6.549378,-11.844045,-1.521764,-5.436530,0.303858,...,-5.716995,-4.017773,-7.356665,-2.826426,-10.775465,-5.283100,-2.942165,-8.829619,-4.390124,1.390651
"'SUNITINIB', 'MK-2206'",7.655789,10.218698,2.884418,1.690343,4.713434,7.130733,12.397303,7.355905,5.837772,9.265481,...,8.397762,0.855489,8.339808,6.867156,9.821957,5.651981,8.458714,7.249948,8.969359,8.201561
"'PACLITAXEL', 'BEZ-235'",14.946517,7.139877,13.032786,-7.144938,4.618305,0.620143,-14.503374,3.262034,7.055163,9.212104,...,-26.277206,0.030083,6.211800,10.507587,-9.994697,6.105050,-0.247124,10.108828,5.658159,16.772867
"'MK-2206', 'DINACICLIB'",-3.198767,3.766895,-40.078484,0.283163,-1.806020,-2.045473,7.799680,2.922471,-5.816652,4.823893,...,6.289625,-0.090478,-6.170962,0.589441,7.788918,-12.455383,3.785094,-4.526407,-2.433933,2.336479
"'MK-4827', 'MK-8776'",-4.906098,10.250538,-43.189118,-8.940379,-2.337250,-1.516254,-2.439822,19.847234,-2.475244,12.847681,...,0.302399,10.432276,-4.557207,5.727460,-2.053051,-2.864524,10.122752,4.894216,5.573604,1.637553
"'VINORELBINE', 'DINACICLIB'",-7.007000,-2.466188,-9.051505,-23.359623,-12.898340,-10.143935,-18.914200,-10.368645,-5.799049,-5.121465,...,-15.584724,-16.806404,-12.472802,-4.164784,-13.014003,-9.905918,-7.013236,-10.443665,-18.329504,-2.320410
"'MK-5108', 'MK-8776'",7.558850,0.274629,-3.063444,14.106067,13.700925,9.097948,26.535269,18.968331,9.326539,19.306631,...,17.780699,-3.634145,6.622278,3.853722,25.757829,12.774137,8.570867,3.428325,18.054030,3.788396


In [25]:
MSE=pow(data222-sens,2).sum().sum()/(sens.shape[0]*sens.shape[1])
MSE

575.1169151942842